# Sending SQuaSH data to InfluxDB

In this example notebook we will sync the SQuaSH production data with our InfluxDB instance, so that we can visualize SQuaSH metrics using [Chronograf](https://chronograf-demo.lsst.codes/). See also [this notebook](https://github.com/lsst-sqre/influx-demo) for a quick introduction on InfluxDB concepts.

We are using the [InfluxDB HTTP API](https://docs.influxdata.com/influxdb/v1.6/tools/api/) for sending the data. The [InfluxDB python client](https://github.com/influxdata/influxdb-python) would also be an option.

In [ ]:
SQUASH_API_URL = "https://squash-restful-api.lsst.codes/"
INFLUXDB_API_URL = "https://influxdb-demo.lsst.codes"

We start by creating a new database. Note that if the database already exists nothing is done (the existing data is preserved), and an status code 200 (OK) is returned.

In [ ]:
import requests
import json

DB = "squash-prod"

params={'q': 'CREATE DATABASE "{}"'.format(DB)}
r = requests.post(url=INFLUXDB_URL + "/query", params=params)
r.status_code

Here we get a list of existing verification jobs from the SQuaSH API.

In [ ]:
jobs = requests.get(SQUASH_API_URL + "/jobs").json()
print("Loading {} verification jobs from SQuaSH...".format(len(jobs['ids'])))

The following cell will actually grab the SQuaSH data and write it in the format used by InfluxDB (called [line protocol](https://docs.influxdata.com/influxdb/v1.6/write_protocols/line_protocol_tutorial/)):


```#<measurement>[,<tag_key>=<tag_value>[,<tag_key>=<tag_value>]] <field_key>=<field_value>[,<field_key>=<field_value>] [<timestamp>]```

the important thing here is that a mesurement is equivalent to a "table" in the database, tags are annotations that are used to query the data, and thus are indexed. Fields correspond to the actual values, and the timestamp acts like the "primary key" in a time series database.

As you run this notebook you might follow the data ingestion using the [Data Explorer](https://chronograf-demo.lsst.codes/sources/2/chronograf/data-explorer) tool in Chronograf.

In [ ]:
from pytz import UTC
from datetime import datetime
from dateutil.parser import parse

EPOCH = UTC.localize(datetime.utcfromtimestamp(0))

params = {'db': DB}

for job_id in jobs['ids']:
    
    r = requests.get(SQUASH_API_URL + "/job/{}".format(job_id)).json()

    # Skip datasets we don't want 
    if r['ci_dataset'] == 'unknown' or r['ci_dataset'] == 'decam':
        continue

    print('Sending line for job {}...'.format(job_id))

    for meas in r['measurements']:

        measurement = meas['metric'].split('.')[0]

        # a tag value cannot have space in them
        tags =  "filter_name={},dataset={}".format(r['meta']['filter_name'], r['ci_dataset'].replace(" ", "_"))

        fields = "{}={}".format(meas['metric'], meas['value'])

        timestamp = int((parse(r['date_created']) - EPOCH).total_seconds()*1e9)

        line = "{},{} {} {}".format(measurement, tags, fields, timestamp)

        print(line)
        post = requests.post(url=INFLUXDB_URL + "/write", params=params, data=line)

        print(post.status_code)
        print(post.text)
